# Probability Loading Benchmark.

$$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$



In [ ]:
import sys
sys.path.append("../")
import matplotlib.pyplot as plt
import numpy as np

## 01. Kernel

The **PL Kernel** can be defined, mathematically as follows:

Let $\mathbf{V}$ be a normalised vector of complex values:

\begin{equation}\label{eq:vector}
    \mathbf{V} = \{v_0, v_1, \cdot, v_{2^n-1} \}, v_i\in \mathbb{C} 
\end{equation}

such that

\begin{equation}\label{eq:vector_norm}
    \sum_{i=0}^{2^n-1}|v_i|^2 =1
\end{equation}

The main task of the **PL Kernel** is the creation of an operator $\mathbf{U}$, from the normalised vector $\mathbf{V}$, which satisfies equation:

\begin{equation}
    \mathbf{U}|0\rangle_n = \sum_{i=0}^{2^n-1} v_i|i\rangle_n
\end{equation}

In the case of the **TNBS** we are going to use a probability density, **pdf**, as the input vector (so $V = P$):

\begin{equation}\label{eq:probabilÇities}
    \mathbf{P} = \{p_0, p_1, \cdot, p_{2^n-1} \}, p_i\in [0,1] 
\end{equation}

where:

\begin{equation}\label{eq:prob_norm}
    \sum_{i=0}^{2^n-1}|p_i|^2 =1
\end{equation}

For this particular case:

\begin{equation}\label{eq:problem_pl2}
    \mathbf{U}_p|0\rangle_n = \sum_{i=0}^{2^n-1} \sqrt{p_i}|i\rangle_n
\end{equation}

## 02. Benchmark Test Case.


The associated **BTC** for the **PL** benchmark will be the loading of a Gaussian function. The procedure will be:

1. Create the discrete probability density function
2. Creating the probability loading unitary operator $\mathbf{U}_p$
3. Execution of the quantum program and measuring of the quantum probability distribution.
4. Metrics computation.

### 1. Create the discrete probability density function.

We need to create the discrete probability density function. The **TNBS** fixes the following procedure:

* Take a random uniform distribution with a particular mean, $\tilde{\mu}$ and standard deviation, $\tilde{\sigma}$, selected within the following ranges:
    * $\tilde{\mu} \in [-2, 2]$
    * $\tilde{\sigma} \in [0.1, 2]$
* So the normal \textbf{PDF} is: $N_{\tilde{\mu},\tilde{\sigma}} (x)$ 
* Set the number of qubits to $n$.
* Create an array of $2^n$ values: $\mathbf{x}=\{x_0, x_1, x_2, \cdots, x_{2^n-1}\}$ where
    * $x_0$ such that $$\int _{-\infty} ^{x_0} N_{\tilde{\mu},\tilde{\sigma}}(x)dx = 0.05$$
    * $x_{2^n-1}$ such that $$\int _{-\infty} ^{x_{2^n-1}}N_{\tilde{\mu},\tilde{\sigma}}(x) dx = 0.95$$
    * $x_{i+1} = x_i + \Delta x$
    * $\Delta x = \frac{x_{2^n-1}-x_0}{2^n}$
* Create a $2^n$ values array, $\mathbf{P}$ from $\mathbf{x}$ by:  
    $$\mathbf{P}(\mathbf{x}) = \{ P(x_0), P(x_1), \cdots, P(x_{2^n-1}) \} = \{N_{\tilde{\mu},\tilde{\sigma}}(x_0), N_{\tilde{\mu},\tilde{\sigma}}(x_1), \cdots, N_{\tilde{\mu},\tilde{\sigma}}(x_{2^n-1}) \}$$
* Normalize the $\mathbf{P}$ array: 
    $$\mathbf{P_{norm}}(\mathbf{x}) = \{ P_{norm}(x_0), P_{norm}(x_1), \cdots, P_{norm}(x_{2^n-1}) \}$$
    where $$P_{norm}(x_{i}) = \frac{P(x_i)}{\sum_{j=0}^{2^n-1} P(x_j)}$$
* Compute the number of shots $n_{shots}$   as:
    $$n_{shots} = \min(10^6, \frac{100}{\min(\mathbf{P_{norm}}(\mathbf{x}))})$$
    
All this part of the procedure is implemented by the *get_theoric_probability* function from **PL/data\_loading** module. The function takes the number of qubits as input and returns  the following outputs:

* $x$ 
* $P_{norm}(x)$
* $\tilde{\mu}$
* $\tilde{\mu}$
* $\Delta x*$
* $n_{shots}$
* $\sum_{j=0}^{2^n-1} P(x_j)$

Each time the function is executed a different distribution will be returned with  $\tilde{\mu} \in [-2, 2]$ and $\tilde{\sigma} \in [0.1, 2]$.

In [ ]:
from data_loading import get_theoric_probability

In [ ]:
x, pn, mu, sigma, deltax, shots, norm = get_theoric_probability(5)
y, pny, muy, sigmay, deltay, shotsy, normy = get_theoric_probability(5)

In [ ]:
plt.plot(x, pn, '-o')
plt.plot(y, pny, '-o')

### 2. Creating the probability loading unitary operator $\mathbf{U}_p$,

Once the discrete probability distribution is created the unitary operator $\mathbf{U}_p$ for loading it into a quantum state should be created. This operator $\mathbf{U}_p$ acts in the following way:

\begin{equation}
    \mathbf{U}_p|0\rangle_n = \sum_{i=0}^{2^n-1} \sqrt{p_i}|i\rangle_n
\end{equation}

The *load_probability* function from **PL/data\_loading** module creates this operator $\mathbf{U}_p$ given the discrete probability function as input array. The function needs 2 inputs:
* array with the normalised discrete probability array
* method: string for selecting the algorithm  for creating the $\mathbf{U}_p$. The algorithm for creating the $\mathbf{U}_p$ will be the one that appeared in: *Grover, L., & Rudolph, T. (2002). Creating superpositions that correspond to efficiently integrable probability distributions*. In this algorithm, controlled rotations by state are needed to load the probability distribution into the quantum state. The selection method allows different implementations of these controlled rotations by state:
    * *brute\_force*: uses the direct implementation of controlled rotation by state.
    * *multiplexor*: the controlled rotations are implemented using **Quantum mulitplexors** as explained in: *V.V. Shende and S.S. Bullock and I.L. Markov. Synthesis of quantum-logic circuits*.
    * *KPTree*: **myqlm** implementation of the *Grover and Rudolph* algorithm  using **Quantum mulitplexors**.
    
The output of the function is a **myqlm** gate with the circuit implementation of the $\mathbf{U}_p$ operator.

In [ ]:
from data_loading import load_probability

In [ ]:
Up_BF = load_probability(pn, "brute_force")
Up_QMF = load_probability(pn, "multiplexor")
Up_KPtree = load_probability(pn, "KPTree")

In [ ]:
%qatdisplay Up_BF --depth 2 --svg

In [ ]:
%qatdisplay Up_QMF --depth 2 --svg

In [ ]:
%qatdisplay Up_KPtree --depth 2 --svg

### 3. Execution of the quantum program and measuring of the quantum probability distribution.

Execute the quantum program $\mathbf{U}|0\rangle_n$ and measure all the $n$ qubits a number of times equal to $n_{shots}$. Store the number of times each state $|i\rangle_n$ is obtained, $m_i$, and compute the probability of obtaining it as $$Q_i = \frac{m_i}{n_{shots}} \forall i = \{0, 1, \cdots, 2^n-1\}$$

This is done by the function *get_qlm_probability* from **data_loading** module. This function executes steps 2 and 3. ´The inputs are:

* array with the normalised discrete probability array
* method: string for selecting the algorithm  for creating the $\mathbf{U}_p$. The algorithm for creating the $\mathbf{U}_p$ will be the one that appeared in *Grover, L., & Rudolph, T. (2002). Creating superpositions that correspond to efficiently integrable probability distributions*. In this algorithm, controlled rotations by state are needed to load the probability distribution into the quantum state. The selection method allows different implementations of these controlled rotations by state:
    * *brute\_force*: uses the direct implementation of controlled rotation by state.
    * *multiplexor*: the controlled rotations are implemented using **Quantum mulitplexors** as explained in: *V.V. Shende and S.S. Bullock and I.L. Markov. Synthesis of quantum-logic circuits*.
    * *KPTree*: **myqlm** implementation of the *Grover and Rudolph* algorithm  using **Quantum mulitplexors**.
* shots: $n_{shots}$ the circuit should be executed and measured.
* qpu: **myqlm** quantum process unit (**QPU**) for executing the computation.

The outputs of the function are:
* result: pandas DataFrame with the results of the measurements by possible state.
* circuit: complete executed circuit in my_qlm format
* quantum_time: time needed for obtaining the complete quantum distribution.
  

In [ ]:
from data_loading import get_qlm_probability

First we need to instanciate the **QPU**. We can use the *get_qpu* from **get_qpu** module (in the **BTC_01_PL** folder)

In [ ]:
sys.path.append("../../")
from get_qpu import get_qpu

In [ ]:
qpu_string = "c" #python, linalg, mps.
# For CESGA Users QLM can be used:
#qpu_string = "qlmass_linalg
qpu = get_qpu(qpu_string)

In [ ]:
result, circuit, qtime = get_qlm_probability(pn, "multiplexor", shots, qpu)

In [ ]:
result

In [ ]:
%qatdisplay circuit --depth  --svg

In [ ]:
print("Time to solution: {}".format(qtime))

### 4. Metrics Computation

Finally, we need to compare the theoretical probability distribution ($P_{norm}$) and the quantum ones ($Q$). This is done using 2 different metrics:

* The Kolmogorov-Smirnov (*KS*)$$KS = \max \left(\left|\sum_{j=0}^i P_{norm}(x_j) - \sum_{j=0}^i Q_j \right|, \; \forall i=0,1,\cdots, 2^n-1 \right)$$
* The Kullback-Leibler divergence (*KL*): $$KL(\mathbf{Q} / \mathbf{P_{norm}}) = P_{norm}(x_j) \ln{\frac{P_{norm}(x_j)}{\max(\epsilon, Q_k)}}$$ where $\epsilon = \min(P_{norm}(x_j)) * 10^{-5}$ which guarantees the logarithm exists when $Q_k=0$
  

In [ ]:
from scipy.stats import entropy

In [ ]:
ks = np.abs(result["Probability"].cumsum() - pn.cumsum()).max()
epsilon = pn.min() * 1.0e-5
kl = entropy(pn, np.maximum(epsilon, result["Probability"]))


print("The Kolmogorov-Smirnov is: {}".format(ks))
print("The Kullback-Leibler divergence is: {}".format(kl))

In [ ]:
plt.plot(x, pn, '-')
plt.plot(x, result["Probability"], 'o')
plt.legend(["theoretical pdf", "quantum pdf"])

## 03. The *LoadProbabilityDensity* class

The *LoadProbabilityDensity* python class inside the **PL/load_probabilities** module allows the user to build the procedure explained in section 02 of the notebook easily and directly. When the class is instantiated a Python dictionary that configures the **BTC** execution should be provided. The mandatory keys are:

* load_method: string with the method for implementing the $\mathbf{U}_p$
* number_of_qbits: number of qubits for discretizing the domain.
* qpu:  string with the **myqlm QPU** for executing the case.

In [ ]:
from load_probabilities import LoadProbabilityDensity

In [ ]:
configuration = {
    "load_method": "brute_force", "number_of_qbits": 8, "qpu": qpu
}
btc_pl = LoadProbabilityDensity(**configuration)

For executing the procedure the *exe* method of the class should be invoked.

In [ ]:
btc_pl.exe()

The following attributes can be accessed:

* data: numoy array with the theoretical pdf.
* result: pandas DataFrame with the quantum pdf
* circuit: *myqlm* circuit
* mean: mean of the theoric Gaussian distribution
* sigma: variance of the theoric Gaussian distribution
* ks: Kolmogorov-Smirnov metric
* kl: Kullback-Leibler divergence

In [ ]:
print("The mean of the Gaussian pdf is: {}".format(btc_pl.mean))
print("The variance of the Gaussian pdf is: {}".format(btc_pl.sigma))

print("The Kolmogorov-Smirnov is: {}".format(ks))
print("The Kullback-Leibler divergence is: {}".format(kl))

In [ ]:
plt.plot(btc_pl.x_, btc_pl.data, '-')
plt.plot(btc_pl.x_, btc_pl.result["Probability"], 'o', alpha=0.3)
plt.legend(["theoretical pdf", "quantum pdf"])

In [ ]:
circuit = btc_pl.circuit
%qatdisplay circuit --dept --svg

Finally the method *summary* creates a pandas DataFrame (*pdf* attribute) with the complete information of the execution


In [ ]:
btc_pl.summary()

In [ ]:
btc_pl.pdf

### Command line execution

The complete **BTC** can be executed by invoking the module **PL/load_probabilities** as a command line. For getting the inputs arguments the following command can be used:

    python load_probabilities.py -h

Example: for a 10 qubits circuit using *KPTree* the following command can be used (c lineal algebra library is used).

    python load_probabilities.py -n_qbits 8 -method KPTree -qpu c

## 04. my\_benchmark\_execution

A complete benchmark execution following the **TNBS** guidelines can be performed by using the **my\_benchmark\_execution.py** module in the **BTC_01_PL** folder.

The probability loading algorithm can be configured in the *kernel_configuration* dictionary at the end of the file. Additionally, the number of qubits for executing the complete benchmark can be provided as a list to the key *list_of_qbits* of the *benchmark_arguments*.

For changing the folder where all the files generated by the benchmark are stored the path can be provided to the key *saving_folder*  of the *benchmark_arguments*.

## 05. Generating the JSON file.

Once the files from a complete benchmark execution are generated the information should be formated following the **NEASQC JSON schema**. For doing this the **neasqc_benchmark.py** module can be used. At the end of the file the path to the folder where all the files from benchmark are stored should be provided to the variable **folder**.

For creating the JSON file following command should eb executed:

    python neasqc_benchmark.py

## 06. Complete Workflow.

The bash script **benchmark_exe.sh** allows to automatize the execution of the benchamrk and the JSON file generation (once the *my_benchmark_execution.py* and the *neasqc_benchmark.py* are properly configured).

    bash benchmark_exe.sh